In [15]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import re
import shutil
from collections import Counter
import string
from tqdm import tqdm
from nltk.tag import map_tag
from gensim.parsing.preprocessing import remove_stopwords, strip_non_alphanum, strip_multiple_whitespaces, strip_punctuation, strip_numeric

In [16]:
train = pd.read_csv("../dataset/goodreads_train.csv")

In [27]:
arch = np.load(file="../vocabulaires/prepro_train_archive_NEG_lem.csv.npy", allow_pickle=True)

In [28]:
x_train = train['review_text']
x_train_rm_st = x_train
y_train = train['rating']

In [29]:
x_train_rm_st = pd.DataFrame(data=arch)

In [30]:
x_train

0         This is a special book. It started slow for ab...
1         Recommended by Don Katz. Avail for free in Dec...
2         A fun, fast paced science fiction thriller. I ...
3         Recommended reading to understand what is goin...
4         I really enjoyed this book, and there is a lot...
                                ...                        
899995    3.5 stars. \n Jenna is a popular YA author and...
899996    This was a quick read for me. I have read a lo...
899997    ** spoiler alert ** \n 3.5 stars. \n This book...
899998    ** spoiler alert ** \n Another fun read from M...
899999    ** spoiler alert ** \n 3.5 stars \n I liked it...
Name: review_text, Length: 900000, dtype: object

In [31]:
x_train_rm_st

,0
0,"special book . started slow first third , midd..."
1,recommended . free : http : //www.audible.com/...
2,"fun , fast paced science fiction thriller . re..."
3,recommended reading understand going middle am...
4,"really enjoyed book , lot recommend . drag lit..."
...,...
899995,3.5 . popular author agent want character firs...
899996,wa quick read . read lot book recently one ver...
899997,* spoiler alert * 3.5 . book sweet inside ! 's...
899998,* spoiler alert another fun read ! 's new assi...


In [40]:
#x_train_rm_st = x_train_rm_st.apply(lambda x: strip_numeric(strip_non_alphanum(strip_multiple_whitespaces(remove_stopwords(strip_punctuation(x))))))

In [41]:
#this function count all words in dataset
def count_dataset(dataset) -> dict:
    test = dataset.apply(lambda x: Counter(x.split()))
    list_of_word = {}
    for el in tqdm(test):
        for word, nb in el.items():
            if not list_of_word.get(word) is None:
                list_of_word[word].append(nb)
            else:
                list_of_word[word] = [nb]
    return list_of_word

In [42]:
words = count_dataset(x_train_rm_st[0])

100%|███████████████████████████████████████████████████████████████████████| 900000/900000 [00:21<00:00, 42711.84it/s]


In [43]:
print(len(words))

525880


In [44]:
# res2 = 0
# for x in range(len(x_train)):
#
#     res2 += len(x_train.iloc[x])
# res2
#
# res = 0
# for x in range(len(x_train_rm_st)):
#
#     res += len(x_train_rm_st.iloc[x])
# res
#494871

In [45]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens= 550000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=None)


In [46]:
print("tokenizing data")
#layer = tf.keras.layers.StringLookup()
vectorize_layer.adapt(x_train_rm_st[0], batch_size=10000)

tokenizing data


In [47]:
# print(len(x_train_rm_st.iloc[0]))
# print(len(x_train.iloc[0]))

In [48]:
len(vectorize_layer.get_vocabulary())

463288

In [49]:
np.save('voc_lemm_without_NP_NEG', np.array(vectorize_layer.get_vocabulary()))

In [51]:
voc = vectorize_layer.get_vocabulary()

In [53]:
voc = [w for w in voc if len(w) > 1]

In [54]:
len(voc)

463250

In [56]:
voc = [w for w in voc if not w.isnumeric()]

In [57]:
len(voc)

453117

In [59]:
np.save('../vocabulaire/voc_lemm_without_NP_NEG.npy', np.array(voc))